In [ ]:
"""
Conversion functions
"""

# Depth and length conversions
def ft_to_m(inputvalue):
    """
    Converts feet to metres.
    Parameters
    ----------
    inputvalue : float
        Input value in feet.
    Returns
    -------
    float
        Returns value in metres.
    """
    return inputvalue * 0.3048

def m_to_ft(inputvalue):
    """
    Converts metres to feet.
    Parameters
    ----------
    inputvalue : float
        Input value in metres.
    Returns
    -------
    float
        Returns value in feet.
    """
    return inputvalue * 3.28084

def ft_to_in(inputvalue):
    """
    Converts feet to inches.
    Parameters
    ----------
    inputvalue : float
        Input value in feet.
    Returns
    -------
    float
        Returns value in inches.
    """
    return inputvalue * 12

def in_to_ft(inputvalue):
    """
    Converts inches to feet.
    Parameters
    ----------
    inputvalue : float
        Input value in inches.
    Returns
    -------
    float
        Returns value in feet.
    """
    return inputvalue / 12

# Velocity / Slowness conversions
def velocity_to_slowness(inputvalue):
    """
    Converts velocity to slowness.
    Parameters
    ----------
    inputvalue : float
        Input value in velocity units.
    Returns
    -------
    float
        Returns value in slowness units.
    """
    return 1000000/inputvalue

def slowness_to_velocity(inputvalue):
    """
    Converts slowness to velocity.
    Parameters
    ----------
    inputvalue : float
        Input value in slowness units.
    Returns
    -------
    float
        Returns value in velocity units.
    """
    return 1000000/inputvalue

# Temperature conversions
def temperature_convert(inputvalue, inputunits, outputunits):
    """
    Converts temperature from one unit to another.
    Parameters
    ----------
    inputvalue : float
        Input temperature value
    inputunits : string
        Input temperature units:
            c = celsius
            f = fahrenheit
            k = kelvin
    outputunits : string
        Output temperature units:
            c = celsius
            f = fahrenheit
            k = kelvin
    Returns
    -------
    float
        Returns temperature value in required units.
    """
    if inputunits.lower() == "c":
        if outputunits.lower() == "f":
            return (9/5)*inputvalue+32
        elif outputunits.lower() == "k":
            return inputvalue + 273.15
        elif outputunits.lower() == "c":
            return inputvalue
    
    if inputunits.lower() == "f":
        if outputunits.lower() == "c":
            return (5/9)*inputvalue-32
        elif outputunits.lower() == "k":
            return (inputvalue-32) * (5/9)+273.15
        elif outputunits.lower() == "f":
            return inputvalue
    
    if inputunits.lower() == "k":
        if outputunits.lower() == "c":
            return inputvalue - 273.15
        elif outputunits.lower() == "f":
            return (inputvalue-273.15)*(9/5)+32
        elif outputunits.lower() == "k":
            return inputvalue
    units = ["k", "c", "f"]
    
    if inputunits.lower() not in units or outputunits.lower() not in units:
        raise Exception("Enter a valid temperature inputunit or outputunit value. Must be: c, f or k")

In [ ]:
"""
Porosity Calculations
"""

from . import miscfuncs

def porosity_density(rhomatrix, rhofluid, rhobulk, limit_result=False, low_limit=0, high_limit=0.6):
    """
    Calculate porosity from a density log
    Parameters
    ----------
    rhomatrix : float
        Matrix density. 
        
        Typical values:
            Sandstone: 2.65 g/cc
            Limestone: 2.71 g/cc
            Dolomite: 2.80 - 2.85 g/cc
    rhofluid : float
        Fluid density.
    rhobulk : float
        Bulk density from log measurements
    limit_result : bool, optional
        Apply limits to the result value.
        By default False
    low_limit : int, optional
        Low limit. If value falls below this limit it will be set to this value. 
        By default 0
    high_limit : float, optional
        High limit. If value falls above this limit it will be set to this value.
        By default: 0.6
    Returns
    -------
        float
        Density porosity value in decimal units.
    """
    porosity = (rhomatrix - rhobulk)/(rhobulk - rhofluid)

    if limit_result is True:
        return miscfuncs.limit_vals(porosity, low_limit, high_limit)
    else:
        return porosity
        
def porosity_sonic(dtmatrix, dtfluid, dtlog, method="wyllie", limit_result=False, low_limit=0, high_limit=0.6):
    """
    Caculate porosity from a sonic log using either the Wyllie-Time Average equation
    or Raymer-Hunt-Gardener equation.
    Parameters
    ----------
    dtmatrix : float
        Matrix slowness.
        Typical values:
            Sandstone: 52-55 us/ft
            Limestone: 47 us/ft
            Dolomite: 43 us/ft
    dtfluid : float
        Fluid slowness.
    dtlog : [type]
        Slowness (DT) from log measurements.
    method : string
        Select a method for calculating sonic porosity:
        "wyllie" - Wyllie Time Average (default)
        "raymer" - Raymer Hunt Gardner
    limit_result : bool, optional
        Apply limits to the result value.
        By default False
    low_limit : int, optional
        Low limit. If value falls below this limit it will be set to this value. 
        By default 0
    high_limit : float, optional
        High limit. If value falls above this limit it will be set to this value.
        By default: 0.6
    Returns
    -------
    float
        Sonic porosity value in decimal units.
    Raises
    ------
    Exception
        Raise an exception if method value is not 1 or 2.
    References
    ----------
    Wyllie, M.R.J., Gregory, A.R., and Gardner, L.W. 1956. Elastic Wave Velocities in Heterogeneous and Porous Media. Geophysics 21 (1): 41–70.
    Raymer, L.L., Hunt, E.R., and Gardner, J.S. 1980. An Improved Sonic Transit Time-to-Porosity Transform, paper P. Trans., 1980 Annual Logging Symposium, SPWLA, 1–12.
    """
    if method == "wyllie": #Wylie Time Average
        porosity = (dtlog - dtmatrix) / (dtfluid - dtmatrix)
    elif method == "raymer": #Raymer Hunt Gardener 
        alpha = (dtmatrix / (2 * dtfluid)) - 1
        porosity = -alpha-((alpha**2 + (dtmatrix / dtlog)-1)**0.5)
    else:
        raise Exception("Enter a valid method value: wyllie or raymer")

    if limit_result is True:
        return miscfuncs.limit_vals(porosity, low_limit, high_limit)
    else:
        return porosity

def porosity_effective(phit, vclay, phitclay):
    """
    Converts total porosity to effective porosity
    Parameters
    ----------
    phit : float
        Total porosity (decima)
    vclay : float
        Volume of clay (decimal)
    phitclay : float
        Clay porosity - taken from a shale interval (decimal)
    Returns
    -------
    float
        Returns effective porosity (decimal)
    """
    return phit - vclay * phitclay

def porosity_total(phie, vclay, phiclay):
    """
    Converts effective porosity to total porosity
    Parameters
    ----------
    phie : float
        Effective porosity (decimal)
    vclay : float
        Volume of clay (decimal)
    phiclay : float
        Clay porosity - taken from a shale interval (decimal)
    Returns
    -------
    float
        Returns total porosity (decimal)
    """
    return phie + vclay * phiclay

def porosity_shale(dens_dry_shale, dens_wet_shale, dens_water):
    """
    Calculates shale porosity.
    Parameters
    ----------
    dens_dry_shale : float
        Dry shale density (g/cc)
    dens_wet_shale : float
        Wet shale density (g/cc)
    dens_water : float
        Water density (g/cc)
    Returns
    -------
    float
        Returns shale porosity (decimal).
    """
    return (dens_dry_shale - dens_wet_shale) / dens_water

In [ ]:
"""
Saturation Calculations
"""

from pypetrophysics import miscfuncs

def formation_factor(arch_a, phi, arch_m):
    """
    Computes Archie Formation Factor (F)
    Parameters
    ----------
    arch_a : float
        Archie Tortuosity Factor - a
    phi : [type]
        Porosity (decimal)
    arch_m : float
        Archie Cementation Exponent - m
    Returns
    -------
    float
        Returns Archie Formation Factor
    """
    return arch_a / phi ** arch_m

def ro(formation_factor, rw):
    """
    Archie Ro - Resistivity of water saturation formation (ohm.m)
    Parameters
    ----------
    formation_factor : float
        Archie Formation Factor
    rw : float
        Resistivity of formation water (ohm.m)
    Returns
    -------
    float
        Returns resistivity of water saturation formation (ohm.m)
    """
    return formation_factor * rw

def resistivity_index(rt, ro):
    """
    Archie Resistivity Index (I)
    Parameters
    ----------
    rt : float
        True formation resistivity (ohm.m)
    ro : float
        Resistivity of water saturated formation (ohm.m)
    Returns
    -------
    float
        Returns Archie resistivity index (I)
    """
    return rt/ro

def sw_archie(phi, rw, rt, arch_a, arch_m, arch_n, limit_result=False, low_limit=0, high_limit=1):
    """
    Archie Water Saturation
    Parameters
    ----------
    phi : float
        Porosity (decimal)
    rw : float
        Water resistivity (ohmm.m)
    rt : float
        True formation resistivity (ohmm.m)
    arch_a : float
        a - Archie Tortuosity Factor
    arch_m : float
        m - Archie Cementation Exponent
    arch_n : float
        n - Archie Saturation Exponent
    limit_result : bool, optional
        Apply limits to the result value.
        By default False
    low_limit : int, optional
        Low limit. If value falls below this limit it will be set to this value. 
        By default 0
    high_limit : float, optional
        High limit. If value falls above this limit it will be set to this value.
        By default: 1
    Returns
    -------
    float
        Returns water saturation computed using the Archie equation in decimal units.
    """
    sw = ((arch_a / phi ** arch_m) * (rw/rt))**(1/arch_n)
    
    if limit_result is True:
        return miscfuncs.limit_vals(sw, low_limit, high_limit)
    else:
        return 